Neste Notebook o dataset dos vistos disponível na internet, pelo link abaixo, e que contem mais de 3 milhões de linhas é modificado para um novo dataset. 

https://www.kaggle.com/nsharan/h-1b-visa

In [2]:
import pandas as pd
import numpy as np
import random
from pandas import ExcelWriter

Na célula abaixo, o dataset é importado e após isso as linhas são embaralhadas. 

In [3]:
dataset = pd.read_csv("h1b.csv")
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset.head()

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,2723275,CERTIFIED,DELOITTE & TOUCHE LLP,Management Analysts,ERS SENIOR CONSULTANT,Y,64355.0,2011.0,"PLYMOUTH, MICHIGAN",-83.470213,42.371425
1,2974243,CERTIFIED,"IDHASOFT, INC.",Operations Research Analysts,OPERATIONS RESERACH ANALYST (SYSTEMS) SR,Y,62317.0,2011.0,"MILPITAS, CALIFORNIA",-121.899574,37.432334
2,2621308,CERTIFIED,UNIVERSITY OF TEXAS MEDICAL BRANCH AT GALVESTON,"Medical Scientists, Except Epidemiologists",RESEARCH COORDINATOR,Y,25875.0,2012.0,"GALVESTON, TEXAS",-94.797696,29.301348
3,2563149,CERTIFIED,DRESSER-RAND COMPANY,Industrial Engineers,INDUSTRIAL ENGINEER,Y,79374.0,2012.0,"OLEAN, NEW YORK",-78.429927,42.083639
4,1288693,CERTIFIED-WITHDRAWN,"SAVEN TECHNOLOGIES, INC.","Software Developers, Applications",COMPUTER SOFTWARE ENGINEER - APPLICATIONS,Y,89981.0,2014.0,"CHICAGO, ILLINOIS",-87.629798,41.878114


Na célula seguinte a um tratamento de dados. Todas as linhas na qual o resultado é WITHDRAWN, são retiradas do dataset, isso pois ele não cai em numa categoria que queira ser estudada, e também pois o processo não foi terminado. As duas linas de código seguintas também retiram linhas do dataset, pelo mesmo motivo a cima, o processo de aplicação para o visto de trabalho não foi terminado. Logo após isso as variavés são transformadas em 0 ou 1, para quando aprovado ou reprovado (CERTIFIED OU DENIED). Vale ressaltar que o caso CERTIFIED-WITHDRAWN, significa que a pessoa foi aprovada mas não quis terminar o processo para obter o visto, esse caso foi considerado como aprovado para o nosso estudo. Por último todas linhas qua não haviam uma das colunas completas, são removidas.

In [4]:
dataset = dataset[dataset.CASE_STATUS != "WITHDRAWN"]
dataset = dataset[dataset.CASE_STATUS != "PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED"]
dataset = dataset[dataset.CASE_STATUS != "INVALIDATED"]

dataset["CASE_STATUS"] = dataset["CASE_STATUS"].apply(lambda x: 1 if (x=="CERTIFIED-WITHDRAWN" or x=="CERTIFIED") else 0)

dataset = dataset.dropna()
dataset.head()

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,2723275,1,DELOITTE & TOUCHE LLP,Management Analysts,ERS SENIOR CONSULTANT,Y,64355.0,2011.0,"PLYMOUTH, MICHIGAN",-83.470213,42.371425
1,2974243,1,"IDHASOFT, INC.",Operations Research Analysts,OPERATIONS RESERACH ANALYST (SYSTEMS) SR,Y,62317.0,2011.0,"MILPITAS, CALIFORNIA",-121.899574,37.432334
2,2621308,1,UNIVERSITY OF TEXAS MEDICAL BRANCH AT GALVESTON,"Medical Scientists, Except Epidemiologists",RESEARCH COORDINATOR,Y,25875.0,2012.0,"GALVESTON, TEXAS",-94.797696,29.301348
3,2563149,1,DRESSER-RAND COMPANY,Industrial Engineers,INDUSTRIAL ENGINEER,Y,79374.0,2012.0,"OLEAN, NEW YORK",-78.429927,42.083639
4,1288693,1,"SAVEN TECHNOLOGIES, INC.","Software Developers, Applications",COMPUTER SOFTWARE ENGINEER - APPLICATIONS,Y,89981.0,2014.0,"CHICAGO, ILLINOIS",-87.629798,41.878114


Abaixo o novos datasets são criados a partir da aprovação ou reprovação no visto e depois são cortados na linha 50000. Por fim esse dois novos dataset são concatenados em um outro, o qual é exportado para um arquivo csv

In [5]:
print(len(dataset))
dataset.head()

2792997


,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,2723275,1,DELOITTE & TOUCHE LLP,Management Analysts,ERS SENIOR CONSULTANT,Y,64355.0,2011.0,"PLYMOUTH, MICHIGAN",-83.470213,42.371425
1,2974243,1,"IDHASOFT, INC.",Operations Research Analysts,OPERATIONS RESERACH ANALYST (SYSTEMS) SR,Y,62317.0,2011.0,"MILPITAS, CALIFORNIA",-121.899574,37.432334
2,2621308,1,UNIVERSITY OF TEXAS MEDICAL BRANCH AT GALVESTON,"Medical Scientists, Except Epidemiologists",RESEARCH COORDINATOR,Y,25875.0,2012.0,"GALVESTON, TEXAS",-94.797696,29.301348
3,2563149,1,DRESSER-RAND COMPANY,Industrial Engineers,INDUSTRIAL ENGINEER,Y,79374.0,2012.0,"OLEAN, NEW YORK",-78.429927,42.083639
4,1288693,1,"SAVEN TECHNOLOGIES, INC.","Software Developers, Applications",COMPUTER SOFTWARE ENGINEER - APPLICATIONS,Y,89981.0,2014.0,"CHICAGO, ILLINOIS",-87.629798,41.878114


In [6]:
na = dataset[dataset["CASE_STATUS"] == 0]
ap = dataset[dataset["CASE_STATUS"] == 1]

size=50000

na_cut = na[:size]
ap_cut = ap[:size]

df = pd.concat([ap_cut, na_cut])
df = df.sample(frac=1).reset_index(drop=True)
df["CASE_STATUS"][df["CASE_STATUS"]==0].size
df.head()

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,1805963,1,"SAGARSOFT, INC",Computer and Information Systems Managers,PROJECT MANAGER,Y,69222.4,2013.0,"SHELTON, CONNECTICUT",-73.093164,41.316486
1,767976,0,MOUNT AUBURN HOSPITAL,PHYSICAL THERAPISTS,PHYSICAL THERAPIST,Y,89648.0,2015.0,"CAMBRIDGE, MASSACHUSETTS",-71.109733,42.373616
2,376800,1,"ERP ANALYSTS, INC.","SOFTWARE DEVELOPERS, APPLICATIONS",SOFTWARE PROGRAMMER/ANALYST,Y,78957.0,2016.0,"CREVE COEUR, MISSOURI",-90.422618,38.660885
3,1233483,1,"ORCHESTRA TECHNOLOGY, INC",NETWORK AND COMPUTER SYSTEMS ADMINISTRATORS,IMS CORE QA TEST ENGINEER,Y,63190.0,2015.0,"BELLEVUE, WASHINGTON",-122.201516,47.610150
4,2235324,0,"SAP INDUSTRIES, INC.",Computer Systems Analysts,APPLICATIONS CONSULTANT IV,Y,97635.0,2012.0,"IRVING, TEXAS",-96.948894,32.814018


In [7]:
df.to_csv("Dataframe.csv")